**Logistic Regression**
- Despite having “regression” in its name, a logistic regression is actually a widely used binary classifier.
$P(y_i=1 \mid X)={\frac{1}{1+e^{-(\beta_{0}+\beta_{1}x)}}}$.
<br>Where: 
- $P(y_i=1 \mid X)$ is the probability of the i-th observation’s target value. 
- $y_i$ is being class 1 
- X is the training data
- $\beta_{0}$ and $\beta_{1}$ are the parameters to be learned
- e is Euler’s number.

In [1]:
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split

In [2]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [3]:
# Standarize features
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

In [4]:
# Create logistic regression object
clf = LogisticRegression(random_state=0)

# Train model
model = clf.fit(X_std, y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [5]:
# Create new observation
new_observation = [[.4, .4, .4, .4]]

In [6]:
# Predict class
model.predict(new_observation)

array([2])

In [7]:
# View predicted probabilities
model.predict_proba(new_observation)

array([[0.12005191, 0.3633989 , 0.51654918]])

**Train Logistic Regression Using SAG solver**

In [8]:
# Create logistic regression object using sag solver
clf_sag = LogisticRegression(random_state=0, solver='sag')

# Train model
model_sag = clf_sag.fit(X_std, y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [9]:
model_sag.predict(new_observation), model_sag.predict_proba(new_observation)

(array([1]), array([[0.06133167, 0.51990398, 0.41876435]]))

**Use Cross-Validation To Find The Best Value Of C** - Fast C Hyperparameter Tuning.
<br>Sometimes the characteristics of a learning algorithm allows us to search for the best hyperparameters significantly faster than either brute force or randomized model search methods. scikit-learn’s LogisticRegressionCV method includes a parameter Cs. If supplied a list, Cs is the candidate hyperparameter values to select from. If supplied a integer, Cs a list of that many candidate values will is drawn from a logarithmic scale between 0.0001 and and 10000 (a range of reasonable values for C).

In [10]:
# Create cross-validated logistic regression
clf_linear = linear_model.LogisticRegressionCV(Cs=100)

# Train model
clf_linear.fit(X, y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


LogisticRegressionCV(Cs=100, class_weight=None, cv='warn', dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='warn', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

**L1 Regularization** (also called least absolute deviations)
<br>is show the effect of the regularization parameter C on the coefficients and model accuracy.

In [11]:
# Remake the variable, keeping all data where the category is not 2.
X_not2 = X[y != 2]
y_not2 = y[y != 2]

In [12]:
X_not2[0:3]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

In [13]:
y_not2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [14]:
X_not2_train, X_not2_test, y_not2_train, y_not2_test = train_test_split(X_not2, y_not2, test_size=0.3, random_state=0)

In [15]:
# Because the regularization penalty is comprised of the sum of the absolute value of the coefficients, we need to scale the data so the coefficients are all based on the same scale.
# Create a scaler object
sc = StandardScaler()

# Fit the scaler to the training data and transform
X_not2_train_std = sc.fit_transform(X_not2_train)

# Apply the scaler to the test data
X_not2_test_std = sc.transform(X_not2_test)

In [16]:
C = [10, 1, 0.1, 0.005, 0.001, 0.0001]

for c in C:
    clf_l1 = LogisticRegression(penalty='l1', C=c, solver='liblinear')
    clf_l1.fit(X_not2_train, y_not2_train)
    print('C:', c)
    print('Coefficient of each feature: ', clf_l1.coef_)
    print('Training accuracy: ', clf_l1.score(X_not2_train_std, y_not2_train))
    print('Test accuracy: ', clf_l1.score(X_not2_test_std, y_not2_test))

C: 10
Coefficient of each feature:  [[-0.09553031 -3.71983564  4.39258717  0.        ]]
Training accuracy:  0.9857142857142858
Test accuracy:  1.0
C: 1
Coefficient of each feature:  [[ 0.         -2.27460208  2.56783887  0.        ]]
Training accuracy:  0.9857142857142858
Test accuracy:  1.0
C: 0.1
Coefficient of each feature:  [[ 0.         -0.8214314   0.97187017  0.        ]]
Training accuracy:  0.9857142857142858
Test accuracy:  1.0
C: 0.005
Coefficient of each feature:  [[0. 0. 0. 0.]]
Training accuracy:  0.5
Test accuracy:  0.5
C: 0.001
Coefficient of each feature:  [[0. 0. 0. 0.]]
Training accuracy:  0.5
Test accuracy:  0.5
C: 0.0001
Coefficient of each feature:  [[0. 0. 0. 0.]]
Training accuracy:  0.5
Test accuracy:  0.5


**One Vs. Rest Logistic Regression**

In [17]:
# Create one-vs-rest logistic regression object
clf_ovr = LogisticRegression(random_state=0, multi_class='ovr')

In [18]:
# Train model
model_ovr = clf_ovr.fit(X_std, y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [19]:
model_ovr.predict(new_observation), model_ovr.predict_proba(new_observation)

(array([2]), array([[0.12005191, 0.3633989 , 0.51654918]]))

**Handling Imbalanced Classes In Logistic Regression**
<br>Like many other learning algorithms in scikit-learn, LogisticRegression comes with a built-in method of handling imbalanced classes. If we have highly imbalanced classes and have no addressed it during preprocessing, we have the option of using the class_weight parameter to weight the classes to make certain we have a balanced mix of each class. Specifically, the balanced argument will automatically weigh classes inversely proportional to their frequency:
$w_j = \frac{n}{kn_{j}}$
where:
- $w_j$ is the weight to class 
- j, n is the number of observations, 
- $n_j$ is the number of observations in class j
- k is the total number of classes.

In [20]:
# Create decision tree classifer object
clf_balanced = LogisticRegression(random_state=0, class_weight='balanced')

# Train model
model_balanced = clf_balanced.fit(X_not2_train_std, y_not2_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [21]:
model_balanced.predict(new_observation), model_balanced.predict_proba(new_observation)

(array([1]), array([[0.24801442, 0.75198558]]))

**Expansion about solver parameter**
[Source](https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-defintions)

A hypothesis h(x), takes an input and gives us the estimated output value.
<br>This hypothesis can be a as simple as a one variable linear equation,... up to a very complicated and long multivariate equation with respect to the type of the algorithm we’re using (i.e. linear regression, logistic regression..etc).
<br><img src="https://i.stack.imgur.com/i8OO5.png"/><br>
- Our task is to find the best Parameters (a.k.a Thetas or Weights) that give us the least error in predicting the output. We call this error a Cost or Loss Function and apparently our goal is to minimize it in order to get the best predicted output!
- One more thing to recall, that the relation between the parameter value and its effect on the cost function (i.e. the error) looks like a bell curve (i.e. Quadratic; recall this because it’s very important) .
- So if we start at any point in that curve and if we keep taking the derivative (i.e. tangent line) of each point we stop at, we will end up at what so called the Global Optima as shown in this image:
<br><img src="https://i.stack.imgur.com/mZ9UU.png"/><br>
If we take the partial derivative at minimum cost point (i.e. global optima) we find the slope of the tangent line = 0 (then we know that we reached our target).
<br>
That’s valid only if we have Convex Cost Function, but if we don’t, we may end up stuck at what so called Local Optima; consider this non-convex function:
<br><img src="https://i.stack.imgur.com/WYEux.png"/><br>
Now you should have the intuition about the hack relationship between what we are doing and the terms: Deravative, Tangent Line, Cost Function, Hypothesis ..etc.
<br>
*Side Note: The above mentioned intuition also related to the Gradient Descent Algorithm (see later)*

**Background**

Linear Approximation:
<br>
Given a function, $f(x)$, we can find its tangent at $x=a$. The equation of the tangent line $L(x)$ is: $L(x)=f(a)+f′(a)(x−a)$.
Take a look at the following graph of a function and its tangent line:
<br><img src="https://i.stack.imgur.com/u0vU0.png"/><br>
From this graph we can see that near $x=a$, the tangent line and the function have nearly the same graph. On occasion we will use the tangent line, $L(x)$, as an approximation to the function, $f(x)$, near $x=a$. In these cases we call the tangent line the linear approximation to the function at $x=a$.

**Quadratic Approximation:**
<br>
Same like linear approximation but this time we are dealing with a curve but we cannot find the point near to 0 by using the tangent line.
<br>
Instead, we use a parabola (which is a curve where any point is at an equal distance from a fixed point or a fixed straight line), like this:
<br><img src="https://i.stack.imgur.com/Yd2mE.png"/><br>
And in order to fit a good parabola, both parabola and quadratic function should have same value, same first derivative, AND second derivative, ...the formula will be (just out of curiosity): $Qa(x) = f(a) + f'(a)(x-a) + \frac{f''(a)(x-a)^2}{2}$
<br>
Now we should be ready to do the comparison in details.

**1. Newton’s Method**

Recall the motivation for gradient descent step at x: we minimize the quadratic function (i.e. Cost Function).

Newton’s method uses in a sense a better quadratic function minimisation. A better because it uses the quadratic approximation (i.e. first AND second partial derivatives).

You can imagine it as a twisted Gradient Descent with The Hessian (The Hessian is a square matrix of second-order partial derivatives of order nxn).

Moreover, the geometric interpretation of Newton's method is that at each iteration one approximates  f(x) by a quadratic function around xn, and then takes a step towards the maximum/minimum of that quadratic function (in higher dimensions, this may also be a saddle point). Note that if f(x) happens to be a quadratic function, then the exact extremum is found in one step.

*Drawbacks:*

1. It’s computationally expensive because of The Hessian Matrix (i.e. second partial derivatives calculations).

2. It attracts to Saddle Points which are common in multivariable optimization (i.e. a point its partial derivatives disagree over whether this input should be a maximum or a minimum point!).

**2. Limited-memory Broyden–Fletcher–Goldfarb–Shanno Algorithm:**

In a nutshell, it is analogue of the Newton’s Method but here the Hessian matrix is approximated using updates specified by gradient evaluations (or approximate gradient evaluations). In other words, using an estimation to the inverse Hessian matrix.

The term Limited-memory simply means it stores only a few vectors that represent the approximation implicitly.

If I dare say that when dataset is small, L-BFGS relatively performs the best compared to other methods especially it saves a lot of memory, however there are some “serious” drawbacks such that if it is unsafeguarded, it may not converge to anything.

**3. A Library for Large Linear Classification:**

It’s a linear classification that supports logistic regression and linear support vector machines (A linear classifier achieves this by making a classification decision based on the value of a linear combination of the characteristics i.e feature value).

The solver uses a coordinate descent (CD) algorithm that solves optimization problems by successively performing approximate minimization along coordinate directions or coordinate hyperplanes.

*LIBLINEAR* is the winner of ICML 2008 large-scale learning challenge. It applies Automatic parameter selection (a.k.a L1 Regularization) and it’s recommended when you have high dimension dataset (recommended for solving large-scale classification problems)

*Drawbacks:*

It may get stuck at a non-stationary point (i.e. non-optima) if the level curves of a function are not smooth.

Also cannot run in parallel.

It cannot learn a true multinomial (multiclass) model; instead, the optimization problem is decomposed in a “one-vs-rest” fashion so separate binary classifiers are trained for all classes.

*Side note: According to Scikit Documentation: The “**liblinear**” solver is used by **default** for historical reasons.*

**4. Stochastic Average Gradient: - SAG**

SAG method optimizes the sum of a finite number of smooth convex functions. Like stochastic gradient (SG) methods, the SAG method's iteration cost is independent of the number of terms in the sum. However, by incorporating a memory of previous gradient values the SAG method achieves a faster convergence rate than black-box SG methods.

It is faster than other solvers for large datasets, when both the number of samples and the number of features are large.

*Drawbacks:*

It only supports **L2 penalization**.
Its memory cost of O(N), which can make it impractical for large N (because it remembers the most recently computed values for approx. all gradients).

**5. SAGA:**

The SAGA solver is a variant of SAG that also supports the non-smooth penalty=l1 option (i.e. L1 Regularization). This is therefore the solver of choice for sparse multinomial logistic regression and it’s also suitable very Large dataset.

*Side note: According to Scikit Documentation: The **SAGA solver** is often the best choice.*

## SUMMARY
<br><img src="https://i.stack.imgur.com/K568D.png"/><br>